# Imports

In [4]:
!pip install cohere --q --disable-pip-version-check

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


# Utils

In [2]:
def print_tool_calls(tool_calls):
    # Determine the message based on the number of tool calls
    if len(tool_calls) > 1:
        print("The model suggests making Parallel Queries:")
    else:
        print("The model suggests making a single tool call:")

    for i, tool_call in enumerate(tool_calls):
        # If there's more than one tool call, separate each with a header
        if len(tool_calls) > 1:
            print(f"== Parallel Tool Call #{i+1}")

        # Print the tool call name and "with this code:" on the same line
        if tool_call.name == 'python_interpreter':
            print(f"{tool_call.name} with this code:")
            code = tool_call.parameters.get('code', '')
            print("\n".join(f"  {line}" for line_num, line in enumerate(code.splitlines())))
        else:
            # For non-python_interpreter tool calls, just print the parameters
            print(f"{tool_call.name}")
            print(f"{tool_call.parameters}")


In [3]:
def print_step(step):
  print(f"\n\n\033[1;34m == STEP {step+1} \033[0m\n")  # Blue bold for step header

def print_plan(text):
  print(f"\033[1;32mThe plan is:\033[0m \033[1m{text}\033[0m\n")  # Green bold for "The plan is" and bold for the plan text

def print_final_answer(text):
  print(f"\n\n \033[1;32mThe final answer is:\033[0m \033[1m{text}\033[0m\n")  # Green bold for "The plan is" and bold for the plan text

def print_tool_results(tool_results):
    print(f"\033[1;35mtool_results:\033[0m \033[1m{tool_result['outputs']}\033[0m")  # Magenta bold for "tool_results" and bold for outputs


# Define Tools for the RAG Agent: a web search engine and a python interpreter

A web search engine, accessible through an API

In [5]:
!pip install tavily-python --q --disable-pip-version-check
from tavily import TavilyClient
tavily_client = TavilyClient(api_key="***")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00


In [6]:
# here's a web search engine
def web_search(query: str) -> list[dict]:
  response = tavily_client.search(query, max_results=3)['results']
  return response

In [7]:
# the LLM is equipped with a description of the web search engine
web_search_tool = {
  "name": "web_search",
  "description": "Returns a list of relevant document snippets for a textual query retrieved from the internet",
  "parameter_definitions": {
    "query": {
      "description": "Query to search the internet with",
      "type": "str",
      "required": True
    }
  }
}


A Python interpreter, which can be used by the RAG Agent to write code and/or to access a spreadsheet

In [8]:
# here's a python console, which can be used to access the spreadsheet, but also more generally to code and plot stuff
import io, contextlib
def python_interpreter(code: str) -> list[dict]:
    output = io.StringIO()
    try:
        # Redirect stdout to capture print statements
        with contextlib.redirect_stdout(output):
            exec(code, globals())
    except Exception as e:
        return [{
            "error": str(e),
            "executed_code": code
        }]
    # Get stdout
    return [{
  		"console_output": output.getvalue(),
      "executed_code": code
  	}]



In [9]:
# the LLM is equipped with a description of a python console
python_interpreter_tool = {
  "name": "python_interpreter",
  "description": "Executes python code and returns the result. The code runs in a static sandbox without internet access and without interactive mode, so print output or save output to a file.",
  "parameter_definitions": {
    "code": {
      "description": "Python code to execute",
      "type": "str",
      "required": True
    }
  }
}


In [11]:
# Display the first few rows of the dataset
import pandas as pd
file_path = './spotify_dataset.csv'
spotify_data = pd.read_csv(file_path)
spotify_data.head(3)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness,release_date
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703.0,43,...,B,Major,80,89,83,31,0,8,4,2023-07-14
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286.0,48,...,C#,Major,71,61,74,7,0,10,4,2023-03-23
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974.0,94,...,F,Major,51,32,53,17,0,31,6,2023-06-30


Utils

In [12]:
functions_map = {
    "web_search": web_search,
    "python_interpreter": python_interpreter,
}

# Cohere model to power Agentic RAG

In [14]:
import cohere
co = cohere.Client(api_key="***")
model = "command-r-08-2024"

# Let's look at a complex user query

In [15]:
message = """What's the age and citizenship of the artists who had the top 3 most streamed songs on Spotify in 2023?

You have access to a dataset with information about Spotify songs from the past 10 years, located at ./spotify_dataset.csv.
You also have access to the internet.
You must use the dataset when you can, and if stuck you can use the internet.
Remember to inspect the dataset to understand its structure before trying to query it. Take it step by step.
"""

### Get the model plan

In [16]:
response = co.chat(
    model=model,
    message=message,
    tools=[web_search_tool,python_interpreter_tool],
    temperature=0,
)

print(response.text)

I will first inspect the dataset to understand its structure and identify the relevant columns for the query. Then, I will write and execute Python code to filter the data for the top 3 most streamed songs on Spotify in 2023 and extract the age and citizenship of the artists.


### Let the model make as many steps of (parallel) tool calling, retrieval, self-reflection, before answering

In [17]:
step = 0

while response.tool_calls:

    # Step id
    print_step(step)

    # The model plan
    print_plan(response.text)

    # Tool calls suggested by the model
    print_tool_calls(response.tool_calls)

    # Execute the tool calls
    tool_results = []
    for tool_call in response.tool_calls:
        outputs = functions_map[tool_call.name](**tool_call.parameters)
        tool_result = {"call": tool_call, "outputs": outputs}
        tool_results.append(tool_result)
        print_tool_results(tool_result['outputs'])

    # Let the model decide the next step: calling more tools? or answering?
    response = co.chat(
        model=model,
        message="",
        chat_history=response.chat_history,
        tools=[web_search_tool,python_interpreter_tool],
        tool_results=tool_results,
        temperature=0,
    )

    step+=1

# print final answer
print_final_answer(response.text)




 == STEP 1 

The plan is: I will first inspect the dataset to understand its structure and identify the relevant columns for the query. Then, I will write and execute Python code to filter the data for the top 3 most streamed songs on Spotify in 2023 and extract the age and citizenship of the artists.

The model suggests making a single tool call:
python_interpreter with this code:
  import pandas as pd
  
  # Load the dataset
  df = pd.read_csv("./spotify_dataset.csv")
  
  # Check the structure of the dataset
  print(df.info())
  print(df.head())
tool_results: [{'console_output': "<class 'pandas.core.frame.DataFrame'>\nRangeIndex: 952 entries, 0 to 951\nData columns (total 25 columns):\n #   Column                Non-Null Count  Dtype  \n---  ------                --------------  -----  \n 0   track_name            952 non-null    object \n 1   artist(s)_name        952 non-null    object \n 2   artist_count          952 non-null    int64  \n 3   released_year         952 non-null 